In [1]:
%env WORKDIR=/tmp/vault
%env VAULT_K8S_NAMESPACE=vault
%env VAULT_HELM_RELEASE_NAME=vault
%env VAULT_SERVICE_NAME=vault-internal 
%env K8S_CLUSTER_NAME=cluster.local 

env: WORKDIR=/tmp/vault
env: VAULT_K8S_NAMESPACE=vault
env: VAULT_HELM_RELEASE_NAME=vault
env: VAULT_SERVICE_NAME=vault-internal
env: K8S_CLUSTER_NAME=cluster.local


In [2]:
%%bash
cat ${WORKDIR}/cluster-keys.json | jq -r ".root_token"

hvs.IjWOxR9d1SAPnTJY1IidfVNE


## Instalando Vault Secret Operator

In [3]:
%%bash
cat ${WORKDIR}/cluster-keys.json | jq -r ".root_token"

hvs.IjWOxR9d1SAPnTJY1IidfVNE


In [4]:
%env VAULT_ADDR=https://127.0.0.1:8200
%env VAULT_TOKEN=hvs.IjWOxR9d1SAPnTJY1IidfVNE
%env VAULT_CACERT=/tmp/vault/vault.ca

env: VAULT_ADDR=https://127.0.0.1:8200
env: VAULT_TOKEN=hvs.IjWOxR9d1SAPnTJY1IidfVNE
env: VAULT_CACERT=/tmp/vault/vault.ca


In [5]:
%%bash
# Role associated to default service account in the app namespace
vault write auth/kubernetes/role/role1 \
    bound_service_account_names=default \
    bound_service_account_namespaces=vault \
    policies=devk8s \
    ttl=24h


Success! Data written to: auth/kubernetes/role/role1


In [6]:
%%bash
# Role associated to default service account in the app namespace
vault write auth/kubernetes/role/role2 \
    bound_service_account_names=default \
    bound_service_account_namespaces=default \
    policies=devk8s \
    ttl=24h

Success! Data written to: auth/kubernetes/role/role2


In [7]:
%%bash
# Policy that will be used to access static secrets
vault policy write devk8s - <<EOF
path "kvv2/*" {
  capabilities = ["read"]
}
EOF


Success! Uploaded policy: devk8s


In [8]:
%%bash
vault secrets enable -path=kvv2 kv-v2

Success! Enabled the kv-v2 secrets engine at: kvv2/


In [9]:
%%bash
vault kv put kvv2/webapp/config username="static-user" password="static-password"

===== Secret Path =====
kvv2/data/webapp/config

======= Metadata =======
Key                Value
---                -----
created_time       2023-05-23T05:15:14.732609634Z
custom_metadata    <nil>
deletion_time      n/a
destroyed          false
version            1


### Instalando el VSO usando helm

In [10]:
%%bash
helm search repo hashicorp/vault-secrets-operator --devel
helm install --namespace $VAULT_K8S_NAMESPACE vault-secrets-operator hashicorp/vault-secrets-operator --version 0.1.0-beta


NAME                            	CHART VERSION	APP VERSION	DESCRIPTION                          
hashicorp/vault-secrets-operator	0.1.0-beta   	0.1.0-beta 	Official Vault Secrets Operator Chart
NAME: vault-secrets-operator
LAST DEPLOYED: Tue May 23 07:16:58 2023
NAMESPACE: vault
STATUS: deployed
REVISION: 1


In [12]:
%%bash
# Stop cell when operator pod is in running state
kubectl get pods -n vault --watch

NAME                                                         READY   STATUS    RESTARTS   AGE
postgresql-dev-0                                             1/1     Running   0          12m
vault-0                                                      1/1     Running   0          17m
vault-1                                                      1/1     Running   0          17m
vault-2                                                      1/1     Running   0          17m
vault-secrets-operator-controller-manager-67cfc56bcd-t2t9b   2/2     Running   0          31s
Error while terminating subprocess (pid=28372): 


In [20]:
%%bash
# Creamos un secreto con la CA de Vault donde la key del secreto tome la forma ca.crt
# Este secreto tiene que estar presente en todos los namespaces donde creemos un VaultConnection CRD
# al ser los secrets un recurso asociado a un namespace
kubectl create secret generic vault-ca \
   -n $VAULT_K8S_NAMESPACE \
   --from-file=ca.crt=${WORKDIR}/vault.ca

   kubectl create secret generic vault-ca \
   -n default \
   --from-file=ca.crt=${WORKDIR}/vault.ca

error: failed to create secret secrets "vault-ca" already exists


secret/vault-ca created


In [18]:
%%bash
## VaultConnection Custom Resource
cat > vso_crd.yaml <<EOF
---
apiVersion: secrets.hashicorp.com/v1alpha1
kind: VaultConnection
metadata:
  namespace: vault
  name: example
spec:
  # required configuration
  # address to the Vault server.
  address: https://vault.vault.svc.cluster.local:8200

  # optional configuration
  # HTTP headers to be included in all Vault requests.
  # headers: []
  # TLS server name to use as the SNI host for TLS connections.
  # tlsServerName: ""
  # skip TLS verification for TLS connections to Vault.
  skipTLSVerify: false
  # the trusted PEM encoded CA certificate chain stored in a Kubernetes Secret
  caCertSecretRef: "vault-ca"

EOF
kubectl apply -f vso_crd.yaml
echo '---'
kubectl describe VaultConnection example -n vault


vaultconnection.secrets.hashicorp.com/example configured
---
Name:         example
Namespace:    vault
Labels:       <none>
Annotations:  <none>
API Version:  secrets.hashicorp.com/v1alpha1
Kind:         VaultConnection
Metadata:
  Creation Timestamp:  2023-05-23T05:26:07Z
  Finalizers:
    vaultconnection.secrets.hashicorp.com/finalizer
  Generation:        4
  Resource Version:  2704
  UID:               be9f26ba-e55e-4999-b5af-1b001d9fd633
Spec:
  Address:             https://vault.vault.svc.cluster.local:8200
  Ca Cert Secret Ref:  vault-ca
  Skip TLS Verify:     false
Status:
  Valid:  true
Events:
  Type     Reason            Age                  From             Message
  ----     ------            ----                 ----             -------
  Warning  VaultClientError  75s (x15 over 116s)  VaultConnection  Failed to construct Vault client: "ca.crt" was empty in the CA secret vault/vault-ha-tls
  Normal   Accepted          0s                   VaultConnection  VaultConnection 

In [22]:
%%bash
## Otro VaultConnection para otro namespace
## VaultConnection Custom Resource
cat > vso_crd_2.yaml <<EOF
---
apiVersion: secrets.hashicorp.com/v1alpha1
kind: VaultConnection
metadata:
  namespace: default
  name: example
spec:
  # required configuration
  # address to the Vault server.
  address: https://vault.vault.svc.cluster.local:8200

  # optional configuration
  # HTTP headers to be included in all Vault requests.
  # headers: []
  # TLS server name to use as the SNI host for TLS connections.
  # tlsServerName: ""
  # skip TLS verification for TLS connections to Vault.
  skipTLSVerify: false
  # the trusted PEM encoded CA certificate chain stored in a Kubernetes Secret
  caCertSecretRef: "vault-ca"

EOF
kubectl apply -f vso_crd_2.yaml
echo '---'
kubectl describe VaultConnection example

vaultconnection.secrets.hashicorp.com/example unchanged
---
Name:         example
Namespace:    default
Labels:       <none>
Annotations:  <none>
API Version:  secrets.hashicorp.com/v1alpha1
Kind:         VaultConnection
Metadata:
  Creation Timestamp:  2023-05-23T05:26:35Z
  Finalizers:
    vaultconnection.secrets.hashicorp.com/finalizer
  Generation:        2
  Resource Version:  2774
  UID:               a0eb4112-252f-4614-bc6f-dac5443c9e6c
Spec:
  Address:             https://vault.vault.svc.cluster.local:8200
  Ca Cert Secret Ref:  vault-ca
  Skip TLS Verify:     false
Status:
  Valid:  false
Events:
  Type     Reason            Age                   From             Message
  ----     ------            ----                  ----             -------
  Normal   Accepted          4m32s                 VaultConnection  VaultConnection accepted
  Warning  VaultClientError  53s (x15 over 2m15s)  VaultConnection  Failed to construct Vault client: Secret "vault-ca" not found


In [23]:
%%bash
cat > vaultauth_crd.yaml <<EOF
---
apiVersion: secrets.hashicorp.com/v1alpha1
kind: VaultAuth
metadata:
  namespace: vault
  name: example
spec:
  # required configuration
  # VaultConnectionRef of the corresponding VaultConnection CustomResource.
  # If no value is specified the Operator will default to the `default` VaultConnection,
  # configured in its own Kubernetes namespace.
  vaultConnectionRef: example
  # Method to use when authenticating to Vault.
  method: kubernetes
  # Mount to use when authenticating to auth method.
  mount: kubernetes
  # Kubernetes specific auth configuration, requires that the Method be set to kubernetes.
  kubernetes:
    # role to use when authenticating to Vault
    role: role1
    # ServiceAccount to use when authenticating to Vault
    # it is recommended to always provide a unique serviceAccount per Pod/application
    serviceAccount: default

  # optional configuration
  # Vault namespace where the auth backend is mounted (requires Vault Enterprise)
  # namespace: ""
  # Params to use when authenticating to Vault
  # params: []
  # HTTP headers to be included in all Vault authentication requests.
  # headers: []
EOF
kubectl apply -f vaultauth_crd.yaml
echo '---'
kubectl describe VaultAuth example -n vault

bash: line 1: default: command not found


vaultauth.secrets.hashicorp.com/example created
---
Name:         example
Namespace:    vault
Labels:       <none>
Annotations:  <none>
API Version:  secrets.hashicorp.com/v1alpha1
Kind:         VaultAuth
Metadata:
  Creation Timestamp:  2023-05-23T05:31:17Z
  Finalizers:
    vaultauth.secrets.hashicorp.com/finalizer
  Generation:        1
  Resource Version:  2978
  UID:               8ad8d22e-fe98-473a-b53f-f2fbcef0bd7f
Spec:
  Kubernetes:
    Role:                      role1
    Service Account:           default
    Token Expiration Seconds:  600
  Method:                      kubernetes
  Mount:                       kubernetes
  Vault Connection Ref:        example
Status:
  Error:  
  Valid:  true
Events:
  Type    Reason    Age   From       Message
  ----    ------    ----  ----       -------
  Normal  Accepted  0s    VaultAuth  Successfully handled VaultAuth resource request


In [24]:
%%bash
cat > vaultauth_crd2.yaml <<EOF
---
apiVersion: secrets.hashicorp.com/v1alpha1
kind: VaultAuth
metadata:
  namespace: default
  name: example2
spec:
  # required configuration
  # VaultConnectionRef of the corresponding VaultConnection CustomResource.
  # If no value is specified the Operator will default to the `default` VaultConnection,
  # configured in its own Kubernetes namespace.
  vaultConnectionRef: example
  # Method to use when authenticating to Vault.
  method: kubernetes
  # Mount to use when authenticating to auth method.
  mount: kubernetes
  # Kubernetes specific auth configuration, requires that the Method be set to kubernetes.
  kubernetes:
    # role to use when authenticating to Vault
    role: role2
    # ServiceAccount to use when authenticating to Vault
    # it is recommended to always provide a unique serviceAccount per Pod/application
    serviceAccount: default

  # optional configuration
  # Vault namespace where the auth backend is mounted (requires Vault Enterprise)
  # namespace: ""
  # Params to use when authenticating to Vault
  # params: []
  # HTTP headers to be included in all Vault authentication requests.
  # headers: []
EOF
kubectl apply -f vaultauth_crd2.yaml
echo '---'
kubectl describe VaultAuth example

bash: line 1: default: command not found


vaultauth.secrets.hashicorp.com/example2 created
---
Name:         example2
Namespace:    default
Labels:       <none>
Annotations:  <none>
API Version:  secrets.hashicorp.com/v1alpha1
Kind:         VaultAuth
Metadata:
  Creation Timestamp:  2023-05-23T05:31:23Z
  Finalizers:
    vaultauth.secrets.hashicorp.com/finalizer
  Generation:        1
  Resource Version:  2989
  UID:               49b0e7d2-4721-4214-bb9b-99e588815d50
Spec:
  Kubernetes:
    Role:                      role2
    Service Account:           default
    Token Expiration Seconds:  600
  Method:                      kubernetes
  Mount:                       kubernetes
  Vault Connection Ref:        example
Status:
  Error:  
  Valid:  true
Events:
  Type    Reason    Age   From       Message
  ----    ------    ----  ----       -------
  Normal  Accepted  0s    VaultAuth  Successfully handled VaultAuth resource request



## CRD Static Secret

In [25]:
%%bash
## Support KVv1 and KVv2
cat > static_secret.yaml <<EOF
---
apiVersion: secrets.hashicorp.com/v1alpha1
kind: VaultStaticSecret
metadata:
  namespace: vault
  name: example
spec:
  vaultAuthRef: example
  mount: kvv2
  type: kv-v2
  name: webapp/config
  refreshAfter: 60s
  destination:
    create: true
    name: static-secret1
EOF
kubectl apply -f static_secret.yaml
echo '---'
kubectl describe VaultStaticSecret example -n vault
echo '---'
sleep 5
kubectl get secret static-secret1 -n vault -o yaml


vaultstaticsecret.secrets.hashicorp.com/example created
---
Name:         example
Namespace:    vault
Labels:       <none>
Annotations:  <none>
API Version:  secrets.hashicorp.com/v1alpha1
Kind:         VaultStaticSecret
Metadata:
  Creation Timestamp:  2023-05-23T05:31:32Z
  Generation:          1
  Resource Version:    3002
  UID:                 26f727f4-d45b-48e1-9b08-c30dbe1ae7dd
Spec:
  Destination:
    Create:          true
    Name:            static-secret1
  Hmac Secret Data:  true
  Mount:             kvv2
  Name:              webapp/config
  Refresh After:     60s
  Type:              kv-v2
  Vault Auth Ref:    example
Events:              <none>
---
apiVersion: v1
data:
  _raw: eyJkYXRhIjp7InBhc3N3b3JkIjoic3RhdGljLXBhc3N3b3JkIiwidXNlcm5hbWUiOiJzdGF0aWMtdXNlciJ9LCJtZXRhZGF0YSI6eyJjcmVhdGVkX3RpbWUiOiIyMDIzLTA1LTIzVDA1OjE1OjE0LjczMjYwOTYzNFoiLCJjdXN0b21fbWV0YWRhdGEiOm51bGwsImRlbGV0aW9uX3RpbWUiOiIwMDAxLTAxLTAxVDAwOjAwOjAwWiIsImRlc3Ryb3llZCI6ZmFsc2UsInZlcnNpb24iOjF9fQ==
  passw

In [26]:
%%bash
## hacemos los mismo esta vez en el default namespace
cat > static_secret.yaml <<EOF
---
apiVersion: secrets.hashicorp.com/v1alpha1
kind: VaultStaticSecret
metadata:
  namespace: default
  name: example
spec:
  vaultAuthRef: example2
  mount: kvv2
  type: kv-v2
  name: webapp/config
  refreshAfter: 60s
  destination:
    create: true
    name: static-secret1
EOF
kubectl apply -f static_secret.yaml
echo '---'
kubectl describe VaultStaticSecret example
echo '---'
sleep 5
kubectl get secret static-secret1 -o yaml

vaultstaticsecret.secrets.hashicorp.com/example created
---
Name:         example
Namespace:    default
Labels:       <none>
Annotations:  <none>
API Version:  secrets.hashicorp.com/v1alpha1
Kind:         VaultStaticSecret
Metadata:
  Creation Timestamp:  2023-05-23T05:31:58Z
  Generation:          1
  Resource Version:    3042
  UID:                 e94c40c5-54ab-466a-aff5-940ae36ec5e0
Spec:
  Destination:
    Create:          true
    Name:            static-secret1
  Hmac Secret Data:  true
  Mount:             kvv2
  Name:              webapp/config
  Refresh After:     60s
  Type:              kv-v2
  Vault Auth Ref:    example2
Events:              <none>
---
apiVersion: v1
data:
  _raw: eyJkYXRhIjp7InBhc3N3b3JkIjoic3RhdGljLXBhc3N3b3JkIiwidXNlcm5hbWUiOiJzdGF0aWMtdXNlciJ9LCJtZXRhZGF0YSI6eyJjcmVhdGVkX3RpbWUiOiIyMDIzLTA1LTIzVDA1OjE1OjE0LjczMjYwOTYzNFoiLCJjdXN0b21fbWV0YWRhdGEiOm51bGwsImRlbGV0aW9uX3RpbWUiOiIwMDAxLTAxLTAxVDAwOjAwOjAwWiIsImRlc3Ryb3llZCI6ZmFsc2UsInZlcnNpb24iOjF9fQ==
  pa

In [30]:
%%bash
kubectl get secrets -A 
echo "***Esto son los dos secretos que hemos sincronizado**"
kubectl get secrets -A | grep "static"

NAMESPACE     NAME                                           TYPE                            DATA   AGE
default       static-secret1                                 Opaque                          3      93s
default       vault-ca                                       Opaque                          1      3m5s
kube-system   bootstrap-token-w963k4                         bootstrap.kubernetes.io/token   6      40m
vault         postgresql-dev                                 Opaque                          2      28m
vault         sh.helm.release.v1.postgresql-dev.v1           helm.sh/release.v1              1      28m
vault         sh.helm.release.v1.vault-secrets-operator.v1   helm.sh/release.v1              1      16m
vault         sh.helm.release.v1.vault.v1                    helm.sh/release.v1              1      33m
vault         static-secret1                                 Opaque                          3      119s
vault         vault-ca                                       O

In [31]:
%%bash
cat > mypod.yaml <<EOF
apiVersion: v1
kind: Pod
metadata:
  name: mypod
  namespace: vault
spec:
  containers:
  - name: mypod
    image: nginx
    volumeMounts:
    - name: foo
      mountPath: "/etc/foo"
      readOnly: true
  volumes:
  - name: foo
    secret:
      secretName: static-secret1
      optional: true
EOF
kubectl apply -f mypod.yaml
sleep 10

# https://developer.hashicorp.com/vault/tutorials/kubernetes/vault-secrets-operator

pod/mypod created
mypod                                                        0/1     Pending   0          0s
mypod                                                        0/1     Pending   0          0s
mypod                                                        0/1     ContainerCreating   0          0s
mypod                                                        1/1     Running             0          10s


In [32]:
%%bash 
kubectl exec mypod -n vault -- cat /etc/foo/username
echo ""
echo "------"
kubectl exec mypod -n vault -- cat /etc/foo/password

static-user
------
static-password

### mostrando que la serviceaccount del Pod no influye (el acceso viene impersonado por Vault-Auth CRD)

In [ ]:
%%bash
cat > mypod3.yaml <<EOF
apiVersion: v1
kind: Pod
metadata:
  name: mypod3
  namespace: vault
  # Using a different service account
spec:
  serviceAccountName: vault
  containers:
  - name: mypod
    image: redis
    volumeMounts:
    - name: foo
      mountPath: "/etc/foo"
      readOnly: true
  volumes:
  - name: foo
    secret:
      secretName: static-secret1
      optional: true
EOF
kubectl apply -f mypod3.yaml

pod/mypod3 created
mypod3                                                       0/1     Pending             0          0s


mypod3                                                       0/1     Pending             0          0s
mypod3                                                       0/1     ContainerCreating   0          0s
mypod3                                                       1/1     Running             0          9s


In [34]:
%%bash 
sleep 10 
kubectl exec   mypod3 -n vault -- cat /etc/foo/username
echo ""
echo "------"
kubectl exec mypod3 -n vault -- cat /etc/foo/password

static-user
------
static-password

## ¿Cómo funciona la autenticación de Kubernetes?

## (Tras crear la dB engine) Dynamic Secrets

In [35]:
%%bash
# Actualizar la políticas con permisos de lectura al path de la database
vault policy write devk8s - <<EOF
path "kvv2/*" {
  capabilities = ["read"]
}
path "database/creds/readonly" {
  capabilities = [ "read"]
}
EOF


cat > dynamic_secret.yaml <<EOF
---
apiVersion: secrets.hashicorp.com/v1alpha1
kind: VaultDynamicSecret
metadata:
  namespace: default
  name: exampled
spec:
  vaultAuthRef: example2
  mount: database
  role: readonly
  destination:
    create: true
    name: dynamic-secret1
EOF

kubectl apply -f dynamic_secret.yaml
echo '---'
sleep 3
kubectl describe VaultDynamicSecret exampled
echo '---'
sleep 5
kubectl get secret dynamic-secret1 -o yaml

Success! Uploaded policy: devk8s
vaultdynamicsecret.secrets.hashicorp.com/exampled created
---
Name:         exampled
Namespace:    default
Labels:       <none>
Annotations:  <none>
API Version:  secrets.hashicorp.com/v1alpha1
Kind:         VaultDynamicSecret
Metadata:
  Creation Timestamp:  2023-05-23T05:35:59Z
  Generation:          1
  Resource Version:    3389
  UID:                 52e7ee5e-74a1-46c9-bb42-a889008f1fae
Spec:
  Destination:
    Create:        true
    Name:          dynamic-secret1
  Mount:           database
  Role:            readonly
  Vault Auth Ref:  example2
Status:
  Last Renewal Time:     1684820159
  Last Runtime Pod UID:  e634e00d-9227-48e8-8784-21b4640e9d99
  Secret Lease:
    Duration:    3600
    Id:          database/creds/readonly/oNbBGt18OW5Ez9f6RwMV7l0E
    Renewable:   true
    Request ID:  e5c75259-5f72-8d5c-a669-9841da0a162e
Events:
  Type    Reason        Age   From                Message
  ----    ------        ----  ----                -------

In [36]:
%%bash
cat > mypod_db.yaml <<EOF
apiVersion: v1
kind: Pod
metadata:
  name: mypoddb
  # Using a different service account
spec:
  containers:
  - name: mypod
    image: redis
    volumeMounts:
    - name: foo
      mountPath: "/etc/foo"
      readOnly: true
  volumes:
  - name: foo
    secret:
      secretName: dynamic-secret1
      optional: true
EOF


# Despliega el POD
kubectl apply -f mypod_db.yaml
# Espera a que despliegue
sleep 10 
# Chequea secretos
kubectl exec mypoddb  -- ls /etc/foo/
echo ""
echo "------"
kubectl exec mypoddb -- cat /etc/foo/_raw

pod/mypoddb created
_raw
password
username

------
{"password":"-PxhZ85ij26EG773JaY1","username":"v-kubernet-readonly-IgaOHBsAP7FsH7t3zh8v-1684820159"}

## Tras crear la PKI engine

In [41]:
%%bash 

# Actualizar la políticas con permisos de lectura al path de la ldap engine
vault policy write devk8s - <<EOF
path "kvv2/*" {
  capabilities = ["read"]
}
path "database/creds/readonly" {
  capabilities = [ "read"]
}
path "pki_int/issue/example-dot-com" {
  capabilities = [ "update"]
}
EOF

cat > pki_secret.yaml <<EOF
---
apiVersion: secrets.hashicorp.com/v1alpha1
kind: VaultPKISecret
metadata:
  namespace: default
  name: pki
spec:
  vaultAuthRef: example2
  mount: pki_int
  name: example-dot-com
  commonName: opus111.example.com
  format: pem
  expiryOffset: 1s
  ttl: 60s
  #namespace: tenant-1
  destination:
    create: true
    name: pki1
EOF



kubectl apply -f pki_secret.yaml
echo '---'
sleep 3
kubectl describe VaultDynamicSecret pki
echo '---'
sleep 5
kubectl get secret pki1 -o yaml

Success! Uploaded policy: devk8s
vaultpkisecret.secrets.hashicorp.com/pki configured
---


Error from server (NotFound): vaultdynamicsecrets.secrets.hashicorp.com "pki" not found


---
apiVersion: v1
data:
  _raw: eyJjYV9jaGFpbiI6WyItLS0tLUJFR0lOIENFUlRJRklDQVRFLS0tLS1cbk1JSURxRENDQXBDZ0F3SUJBZ0lVUS9ySnB3NUJSYXE1ZVl6QzBJMHdkSktVdzVZd0RRWUpLb1pJaHZjTkFRRUxcbkJRQXdGakVVTUJJR0ExVUVBeE1MWlhoaGJYQnNaUzVqYjIwd0hoY05Nak13TlRJek1EVXdPVFV6V2hjTk1qZ3dcbk5USXhNRFV4TURJeldqQXRNU3N3S1FZRFZRUURFeUpsZUdGdGNHeGxMbU52YlNCSmJuUmxjbTFsWkdsaGRHVWdcblFYVjBhRzl5YVhSNU1JSUJJakFOQmdrcWhraUc5dzBCQVFFRkFBT0NBUThBTUlJQkNnS0NBUUVBcnRYUFNqSzdcbkljbUl1eXlTWGdUeGdITWdraEFjTW5MN2Q5ZlBiK3h6c2cvOWJXSnVtQ0FaUkU1U28vK2Y0eU9Iek55ODBPL0pcbkRUcmQvMXY4UWJGa0pnd3JweE9ZV2VKSm02VEFGSlJWdlluRzJSWUxpZjhMNTk5VjZaZlhwL2xzWitlYlNMcFBcbkl6UGFBd3l5Q1Z2ZWkzQy9tS2xwTFQ4cGVLZFBuNzM4SUt6Q3pWV2NzeTlaSGtyZ1BUaTNPVGI4OUk1dDRoYW1cbkM0REREMWtvZ2MwOXpTcE1YKzFkYnU2WXJEcHcrRUFjV0NxRkZFeGkrc1pGTkFHMW0rNXF0VHBNZEVaR2xIanRcbmpsYW9vSUlicDJ3elNndFdoUmdQcmNwT21JWE02b3BRVWI2K2laZ2NPbnJFdnlzYmNXSTVhMjJBZG16Q3orTThcbjFoYTNKcHZpTDJpdHJRSURBUUFCbzRIV01JSFRNQTRHQTFVZER3RUIvd1FFQXdJQkJqQVBCZ05WSFJNQkFmOEVcbkJUQURBUUgvTUIwR0ExVWREZ1FXQkJ

In [42]:
%%bash
cat > mypod_pki.yaml <<EOF
apiVersion: v1
kind: Pod
metadata:
  name: mypodpki
  # Using a different service account
spec:
  containers:
  - name: mypod
    image: redis
    volumeMounts:
    - name: foo
      mountPath: "/etc/foo"
      readOnly: true
  volumes:
  - name: foo
    secret:
      secretName: pki1
      optional: true
EOF
kubectl apply -f mypod_pki.yaml

pod/mypodpki created


In [43]:
%%bash 
sleep 10 
kubectl exec mypodpki  -- ls /etc/foo/
echo ""
echo "------"
kubectl exec mypodpki -- cat /etc/foo/_raw

_raw
ca_chain
certificate
expiration
issuing_ca
private_key
private_key_type
serial_number

------
{"ca_chain":["-----BEGIN CERTIFICATE-----\nMIIDqDCCApCgAwIBAgIUQ/rJpw5BRaq5eYzC0I0wdJKUw5YwDQYJKoZIhvcNAQEL\nBQAwFjEUMBIGA1UEAxMLZXhhbXBsZS5jb20wHhcNMjMwNTIzMDUwOTUzWhcNMjgw\nNTIxMDUxMDIzWjAtMSswKQYDVQQDEyJleGFtcGxlLmNvbSBJbnRlcm1lZGlhdGUg\nQXV0aG9yaXR5MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEArtXPSjK7\nIcmIuyySXgTxgHMgkhAcMnL7d9fPb+xzsg/9bWJumCAZRE5So/+f4yOHzNy80O/J\nDTrd/1v8QbFkJgwrpxOYWeJJm6TAFJRVvYnG2RYLif8L599V6ZfXp/lsZ+ebSLpP\nIzPaAwyyCVvei3C/mKlpLT8peKdPn738IKzCzVWcsy9ZHkrgPTi3OTb89I5t4ham\nC4DDD1kogc09zSpMX+1dbu6YrDpw+EAcWCqFFExi+sZFNAG1m+5qtTpMdEZGlHjt\njlaooIIbp2wzSgtWhRgPrcpOmIXM6opQUb6+iZgcOnrEvysbcWI5a22AdmzCz+M8\n1ha3JpviL2itrQIDAQABo4HWMIHTMA4GA1UdDwEB/wQEAwIBBjAPBgNVHRMBAf8E\nBTADAQH/MB0GA1UdDgQWBBT0EDOcqiYu5csNEVroMK6QxaFc9TAfBgNVHSMEGDAW\ngBTmgVCiqcHmnSH0/fq+4eRdB7ep+TA8BggrBgEFBQcBAQQwMC4wLAYIKwYBBQUH\nMAKGIGh0dHBzOi8vMTI3LjAuMC4xOjgyMDAvdjEvcGtpL2NhMDIGA1UdHwQrMCkw\n